In [1]:
import pandas as pd
import numpy as np
import csv
from collections import defaultdict

In [2]:
df = pd.read_csv("deliveries.csv")

In [3]:
'''
df1 and df2 can be combined to see the 4s and 6s together if needed. 
you just need to copy the total_runs column from df2 to df1
'''
#has all the 4s for a team in each over all the seasons
df1 = pd.DataFrame(df[df.total_runs == 4].groupby(["batting_team","over"]).count()["total_runs"])
df1.to_csv("perteam4s.csv")
#has all the 6s for a team in each over all the seasons
df2 = pd.DataFrame(df[df.total_runs == 6].groupby(["batting_team","over"]).count()["total_runs"])
df2.to_csv("perteam6s.csv")
#has all the 4s per over for all the teams combined
df4 = pd.DataFrame(df[df.total_runs == 4].groupby(["over"]).count()["total_runs"])
df4.to_csv("perover4s.csv")
#has all the 6s for all the teams combined
df5 = pd.DataFrame(df[df.total_runs == 6].groupby(["over"]).count()["total_runs"])
df5.to_csv("perover6s.csv")

'''
Extas provided by the bowling team
'''
#extras given by the bowling team per over
df6 = pd.DataFrame(df[df.extra_runs > 0].groupby(["bowling_team","batting_team","over"]).count()["total_runs"])
df6.to_csv("perteamextrasbybowlingteam.csv")
#extras given by the bowling team over all
df3 = pd.DataFrame(df[df.extra_runs > 0].groupby(["bowling_team","batting_team"]).count()["total_runs"])
df3.to_csv("perteamextrasbybowlingteam_agg.csv")



In [4]:
dfm = pd.DataFrame(pd.read_csv("matches.csv"))
dfm.head(5)

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [5]:
'''
finding the competing teams, who won the toss and who won the game
'''
for i in range(2008,2018):
    query = "season==" +str(i)
    filename = "winners"+ str(i) + "permatch.csv"
    dfm1 = pd.DataFrame(dfm.query(query)[["team1","team2","toss_winner","toss_decision","winner"]])
    dfm1.to_csv(filename)
    

In [6]:
'''
finding the count of matches for which a player got player of the match
'''
dfm2 = dfm.groupby("player_of_match").count()[["id"]].sort_values("id", ascending = False)
dfm2.to_csv("palyerofthematch.csv")

In [3]:
'''
Runs scored by a batsman

'''
player = defaultdict(int)
player_runs = defaultdict(lambda: defaultdict(int))

for i,row in df.iterrows():
    player[row["batsman"]] += (row["total_runs"]-row["extra_runs"])
    player_runs[row["batsman"]][row["total_runs"]-row["extra_runs"]]+=1
    player_runs[row["batsman"]][-1]+=1

In [4]:
player_runs

defaultdict(<function __main__.<lambda>>,
            {'A Ashish Reddy': defaultdict(int,
                         {-1: 196, 0: 61, 1: 83, 2: 20, 3: 1, 4: 16, 6: 15}),
             'A Chandila': defaultdict(int, {-1: 7, 0: 3, 1: 4}),
             'A Chopra': defaultdict(int, {-1: 75, 0: 45, 1: 21, 2: 2, 4: 7}),
             'A Choudhary': defaultdict(int,
                         {-1: 20, 0: 4, 1: 13, 2: 1, 4: 1, 6: 1}),
             'A Flintoff': defaultdict(int,
                         {-1: 57, 0: 24, 1: 23, 2: 2, 3: 1, 4: 5, 6: 2}),
             'A Kumble': defaultdict(int, {-1: 49, 0: 24, 1: 21, 2: 1, 4: 3}),
             'A Mishra': defaultdict(int,
                         {-1: 382, 0: 168, 1: 162, 2: 20, 4: 27, 6: 5}),
             'A Mithun': defaultdict(int,
                         {-1: 26, 0: 11, 1: 8, 2: 2, 4: 4, 6: 1}),
             'A Mukund': defaultdict(int, {-1: 23, 0: 9, 1: 11, 2: 2, 4: 1}),
             'A Nehra': defaultdict(int,
                         {-1: 63, 0

In [14]:
with open("batsmanruns.csv","w") as f:
    f.write("{},{},{},{},{},{},{},{},{}\n".format("player","runs","0","1","2","3","4","6","-1"))
    for x,y in sorted(player.items(), key=lambda x:x[1],reverse=True):
        runs = player_runs[x]
        print(runs)
        f.write("{},{},{},{},{},{},{},{},{}\n".format(x,y,runs[0],runs[1],runs[2],runs[3],runs[4],runs[6],runs[-1]))

defaultdict(<class 'int'>, {0: 1163, -1: 3369, 1: 1377, 4: 402, 2: 242, 6: 174, 3: 10, 5: 1})
defaultdict(<class 'int'>, {0: 1287, -1: 3494, 1: 1410, 6: 160, 2: 242, 4: 384, 3: 11})
defaultdict(<class 'int'>, {0: 1185, -1: 3274, 1: 1379, 3: 5, 2: 177, 4: 354, 6: 173, 5: 1})
defaultdict(<class 'int'>, {1: 1313, -1: 3433, 0: 1318, 4: 484, 2: 245, 6: 58, 3: 15})
defaultdict(<class 'int'>, {0: 1124, -1: 2902, 4: 401, 6: 160, 1: 997, 2: 207, 3: 13})
defaultdict(<class 'int'>, {0: 1165, -1: 2960, 4: 377, 6: 125, 2: 191, 1: 1087, 3: 12, 5: 3})
defaultdict(<class 'int'>, {1: 718, -1: 2532, 0: 1178, 4: 297, 6: 266, 2: 70, 3: 3})
defaultdict(<class 'int'>, {0: 1210, -1: 3005, 1: 1137, 4: 401, 2: 168, 6: 71, 3: 16, 5: 2})
defaultdict(<class 'int'>, {1: 1085, -1: 2680, 0: 926, 4: 251, 6: 156, 2: 251, 3: 11})
defaultdict(<class 'int'>, {4: 287, -1: 2402, 0: 778, 6: 158, 1: 980, 2: 187, 3: 12})
defaultdict(<class 'int'>, {1: 1039, -1: 2602, 0: 1001, 4: 320, 6: 60, 2: 168, 3: 14})
defaultdict(<class 

In [5]:
sorted(player.items(), key=lambda x:x[1],reverse=True)

[('SK Raina', 4548),
 ('V Kohli', 4423),
 ('RG Sharma', 4207),
 ('G Gambhir', 4132),
 ('DA Warner', 4014),
 ('RV Uthappa', 3778),
 ('CH Gayle', 3651),
 ('S Dhawan', 3561),
 ('MS Dhoni', 3560),
 ('AB de Villiers', 3486),
 ('AM Rahane', 3057),
 ('YK Pathan', 2922),
 ('KD Karthik', 2903),
 ('BB McCullum', 2755),
 ('V Sehwag', 2728),
 ('SR Watson', 2628),
 ('Yuvraj Singh', 2591),
 ('M Vijay', 2511),
 ('SE Marsh', 2489),
 ('JH Kallis', 2427),
 ('AT Rayudu', 2416),
 ('DR Smith', 2385),
 ('KA Pollard', 2354),
 ('SR Tendulkar', 2334),
 ('PA Patel', 2322),
 ('MK Pandey', 2223),
 ('R Dravid', 2174),
 ('AC Gilchrist', 2069),
 ('JP Duminy', 1993),
 ('MEK Hussey', 1977),
 ('DPMD Jayawardene', 1808),
 ('RA Jadeja', 1732),
 ('SPD Smith', 1713),
 ('KC Sangakkara', 1687),
 ('MK Tiwary', 1648),
 ('AJ Finch', 1604),
 ('DA Miller', 1563),
 ('WP Saha', 1557),
 ('NV Ojha', 1553),
 ('S Badrinath', 1441),
 ('SV Samson', 1426),
 ('BJ Hodge', 1400),
 ('SC Ganguly', 1349),
 ('DJ Hussey', 1322),
 ('F du Plessis',

In [15]:
test = pd.read_csv("batsmanruns.csv")

In [16]:
test.head(100)

,player,runs,0,1,2,3,4,6,-1
0,SK Raina,4548,1163,1377,242,10,402,174,3369
1,V Kohli,4423,1287,1410,242,11,384,160,3494
2,RG Sharma,4207,1185,1379,177,5,354,173,3274
3,G Gambhir,4132,1318,1313,245,15,484,58,3433
4,DA Warner,4014,1124,997,207,13,401,160,2902
5,RV Uthappa,3778,1165,1087,191,12,377,125,2960
6,CH Gayle,3651,1178,718,70,3,297,266,2532
7,S Dhawan,3561,1210,1137,168,16,401,71,3005
8,MS Dhoni,3560,926,1085,251,11,251,156,2680
9,AB de Villiers,3486,778,980,187,12,287,158,2402
